27/09/2022

# Add colored segmentation on video

Use this script to add colored annotations and predictions on given dataset movies.

Used to generate examples for midterm exams.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import datetime
import imageio
import itertools

import numpy as np
import math
from PIL import Image
from scipy import spatial, optimize

from data_processing_tools import get_argmax_segmentation_otsu
from in_out_tools import load_annotations_ids,load_movies_ids,load_predictions_ids
from in_out_tools import write_colored_class_videos_on_disk

## Load movies, annotations & training predictions

Predictions need to be first saved running `load_trainings_predict.py`.

In [3]:
# assuming these are movie IDs whose predictions are available...
movie_ids_train = ["01","02","03","04","06","07","08","09",
                            "11","12","13","14","16","17","18","19",
                            "21","22","23","24","27","28","29",
                            "30","33","35","36","38","39",
                            "41","42","43","44","46"]

movie_ids_test  = ["05","10","15","20","25","32","34","40","45"]

movie_ids = movie_ids_train+movie_ids_test

In [4]:
training_name = 'TEMP_new_annotated_peaks_physio'
epoch = 100000

In [5]:
data_dir = os.path.join("..","data","sparks_dataset")

preds_dir = os.path.join("trainings_validation", 
                        training_name)

out_dir = os.path.join("trainings_validation", 
                        training_name, 
                        "colored_segmentation")
os.makedirs(out_dir, exist_ok=True)

#### **Remark:** in case, can also implement adding colored event instances to movies

In [6]:
### Load movies
xs = load_movies_ids(data_folder=data_dir,
                     ids=movie_ids,
                     names_available = True,
                     movie_names = 'video')

### Load annotations -> using current annotations in dataset directory
ys = load_annotations_ids(data_folder=data_dir,
                          ids=movie_ids,
                          mask_names="class_label")

### Load annotated event instances
#annotated_events = load_annotations_ids(data_folder=data_dir,
#                                        ids=movie_ids,
#                                        mask_names="event_label"
#                                        )

### Load predictions
# Predictions created from selected model
sparks_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_sparks.tif") for movie_id in movie_ids}
puffs_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_puffs.tif") for movie_id in movie_ids}
waves_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_waves.tif") for movie_id in movie_ids}

sparks = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in sparks_filenames.items()}
puffs = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in puffs_filenames.items()}
waves = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in waves_filenames.items()}

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


## Get predictions segmentation

In [7]:
preds = {}

In [8]:
for movie_id in movie_ids:
    raw_preds = {'sparks': sparks[movie_id],
                 'puffs': puffs[movie_id],
                 'waves': waves[movie_id],
                 'background': 1-sparks[movie_id]-puffs[movie_id]-waves[movie_id]
                }
    preds[movie_id] = get_argmax_segmentation_otsu(preds=raw_preds,
                                                   get_classes=False,
                                                   debug=False)

## Add segmentation to original movies

### add transparent segmentation on original movies

In [9]:
# parameters
ignore_frames = 6
transparency = 70

In [10]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id
    
    # write predictions on disk
    write_colored_class_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=preds[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_preds",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=False)
    
    # write labels on disk
    write_colored_class_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=ys[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_labels",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=False)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video

### create masks with colored segmentation and white background

In [11]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id+"_white_bg"
    
    # write predictions on disk
    write_colored_class_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=preds[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_preds",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=True)
    
    # write labels on disk
    write_colored_class_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=ys[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_labels",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=True)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video

### save original movie in RGB without ignored frames for comparison

In [12]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    # convert movie to rgb and save on disk
    movie_rgb = np.copy(xs[movie_id])
    movie_rgb = 255*(movie_rgb/movie_rgb.max())
    movie_rgb = [Image.fromarray(frame).convert('RGB') for frame in movie_rgb]
    if ignore_frames > 0:
        movie_rgb = [np.array(frame) 
                     for frame in movie_rgb[ignore_frames:-ignore_frames]]
    else:
        movie_rgb = [np.array(frame) for frame in movie_rgb]
    
    imageio.volwrite(os.path.join(out_dir, movie_id+"_original_movie.tif"),
                     movie_rgb)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video